# Single Layer (1 hidden, 1 output) Diagonal ReLU Recurrent Network (Pachitariu 2013) to sMNIST/rowMNIST

## Organisation 

<ol type="a">
    <li>ReLU RNN (from paper)</li>
    <li>Smooth ReLU RNN (from paper)</li>
    <li>Skip RNN (from paper)</li>
    <li>Diagonal reccurent weights ReLU RNN (my intermediary step) </li>
    <li><p style="font-weight:bold;">Diagonal reccurent weights Linear RNN (from paper, named IRLM) <font color='red'>THIS NOTEBOOK</font></p></li>
    <li>Multi layer: Diagonal reccurent weights (my extension)</li>
</ol>

## Equations
The recurrent weights are now diagonal matrices, which means we can do hadamard products instead of matrix multiplications. These weights are sampled uniformly between -0.999 and 0.999 to ensure that the eigenvalues are less than 1. 

The input and output weights are still dense matrices sampled using LeCun normal initialization.

All biases are initialized to zero.

Initialization:

$
W_{hx} \in \mathbb{R}^{n_h \times n_x} \text{ with } W_{hx} \text{ sampled } \sim \mathcal{N}(0, \sqrt{\frac{1}{n_x}}) \\
\vec{w}_{hh} \in \mathbb{R}^{n_h} \text{ with } \vec{w}_{hh} \text{ sampled } \sim \mathcal{U}(0.999, -0.999) \\
\vec{b}_h \in \mathbb{R}^{n_h} \\
W_{hy} \in \mathbb{R}^{n_y \times n_h} \text{ with } W_{hy} \text{ sampled } \sim \mathcal{N}(0, \sqrt{\frac{1}{n_h}}) \\
\vec{b}_o \in \mathbb{R}^{n_y} \\
$


State update:

$
\vec{h}_t = W_{hx}\vec{x}_t + \vec{w}_{hh}\odot\vec{h}_{t-1} + \vec{b}_h \text{ with }\vec{h}_t \in \mathbb{R}^{n_h} \text{ and } \vec{x}_t \in \mathbb{R}^{n_x} \\
$

Output: mean of output over time (or cummulative sum of output over time):

$
\vec{o}_t = W_{hy}\vec{h}_t + \vec{b}_o \\
\vec{o} = \frac{1}{T}\sum_{t=1}^{T}\vec{o}_t \\
\vec{y} = \text{softmax}(\vec{o}) \\
$

<font color='green'>THINGS SOLVED</font>
- exploding activations --> change the recurrent weight initialization to ensure |w| < 1 

<font color='red'>THINGS TO UNDERSTAND</font>
- why init in glorot using (256,) does not work? --> my own init function
- understand why getting the params in the layers doesn't work
- understand why not running on GPU


In [1]:
!pwd
!which python
!pip freeze | grep -E 'flax|jax|orbax|optax'

/home/tristan/LearningJAX/Flax
/home/tristan/miniconda3/envs/.jax_conda_env_LearningJAX/bin/python
flax==0.8.0
jax==0.4.25
jaxlib==0.4.25+cuda11.cudnn86
optax==0.1.8
orbax-checkpoint==0.5.0


In [1]:
import jax
import numpy as np
import torch
from jax import numpy as jnp
from torch.utils.data import TensorDataset
from tqdm import tqdm
import torchvision
import torchvision.transforms as transforms
from flax import linen as nn
from flax.training import train_state
import optax
from jax.nn.initializers import lecun_normal
from typing import Any, Tuple, Sequence, Optional
import matplotlib.pyplot as plt
px = 1 / plt.rcParams['figure.dpi']

jnp.set_printoptions(precision=3, suppress=True)

# set cuda visible devices
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
from utils import create_mnist_classification_dataset, plot_dynamics

In [3]:
DATASET_VERSION = "sequential" # "row"
trainloader, testloader, N_CLASSES, SEQ_LENGTH, IN_DIM = create_mnist_classification_dataset(root="../data", version=DATASET_VERSION)

[*] Generating MNIST Classification Dataset...


In [4]:
batch_x, batch_y = next(iter(testloader))
print(batch_x.shape, batch_y.shape)
print(batch_y.dtype)

(128, 784, 1) (128,)
int64


In [5]:
from typing import Sequence

def rec_initializer():
    def init(key, shape):
        return jax.random.uniform(key, shape, minval=-0.999, maxval=0.999)
    return init

class IRRNCell(nn.Module):
    # hidden_size: int
    # out_size: int

    def setup(self):
        # self.Linear_in = nn.Dense(self.hidden_size, name='Whx')
        self.Linear_in = nn.Dense(512, name='Whx')
        # self.wh = self.param('wh', lecun_normal(), (self.hidden_size,))
        self.wh = self.param('wh', rec_initializer(), (2,512))
        self.wh = self.wh[0]
        # self.Linear_out = nn.Dense(self.out_size, name='Woh')
        self.Linear_out = nn.Dense(10, name='Woh')


    def __call__(self, state, x):
        # Wh @ h + Wx @ x + b can be efficiently computed
        # by concatenating the vectors and then having a single dense layer
        xb = self.Linear_in(x)
        decayed_state = self.wh * state
        new_state = xb + decayed_state
        out = self.Linear_out(new_state)
        return new_state, (new_state, out)
    
class SimpleScan(nn.Module):
    # hidden_size: int
    # out_size: int

    @nn.compact
    def __call__(self, state, x):
        IRRN = nn.scan(
            IRRNCell,
            variable_broadcast='params',
            split_rngs={'params': False},
            in_axes=0,
            out_axes=0,
        )
        return IRRN()(state, x)
    
class RNNLayer(nn.Module):
    hidden_size: int
    out_size: int

    def setup(self):
        self.SimpleScan = SimpleScan()


    def __call__(self, x):
        state = jnp.zeros((self.hidden_size,))
        
        _, (state_hist, out_hist) = self.SimpleScan(state, x)
        return state_hist, out_hist

class RNN(nn.Module):
    hidden_size: int
    output_size: int
    n_layers: int

    def setup(self):
        self.rnn_cells = [RNNLayer(self.hidden_size, self.output_size) for _ in range(self.n_layers)]
    def __call__(self, x):
        # print(x.shape)
        all_state_hist = []
        for id_layer, rnn_cell in enumerate(self.rnn_cells[:-1]):
            x, _ = rnn_cell(x)
            all_state_hist.append(x)
        state_hist, out_hist = self.rnn_cells[-1](x)
        all_state_hist.append(state_hist)
        return all_state_hist, out_hist
    
BatchRNN = nn.vmap(RNN, in_axes=0, out_axes=0, variable_axes={'params': None}, split_rngs={'params': False})


In [6]:
srn = BatchRNN(512, 10, 1)
params = srn.init(jax.random.PRNGKey(0), jnp.zeros_like(batch_x[:5]))
all_state_hist, out_hist = srn.apply(params, batch_x[:5])
print(len(all_state_hist), [all_state_hist[i].shape for i in range(len(all_state_hist))], out_hist.shape) # (batch, time, cell_size)
y = out_hist.mean(axis=1)
print(y.shape)
assert y.shape == (5, 10)
print(y)

1 [(5, 784, 512)] (5, 784, 10)
(5, 10)
[[ -4.01  -15.639  -7.746  -2.273  -5.093  -6.671  -7.301   2.637  -6.853
   -9.467]
 [ -3.45  -13.461  -6.663  -2.007  -4.403  -5.72   -6.246   2.233  -5.827
   -8.151]
 [ -4.417 -17.245  -8.525  -2.511  -5.621  -7.356  -8.058   2.904  -7.561
  -10.431]
 [ -3.002 -11.705  -5.773  -1.78   -3.856  -4.973  -5.432   1.924  -5.043
   -7.091]
 [ -3.926 -15.315  -7.563  -2.258  -5.009  -6.53   -7.147   2.563  -6.687
   -9.269]]


In [7]:
one_hot = jax.nn.one_hot(batch_y[:5], 10)
loss = optax.softmax_cross_entropy(logits=y, labels=one_hot)
print(loss.shape)
print(loss)
print(loss.mean())


(5,)
[ 0.009  8.916 20.155  4.963  7.583]
8.32517


In [8]:
# print(params['params'].keys())
# print(params['params']['RNNLayer_0'].keys())
# print(params['params']['RNNLayer_0']['ScanRNNCell_0'].keys())

print('\nparams: \n', jax.tree_util.tree_map(jnp.shape, params['params']))

# assert params['params']['RNNLayer_0']['ScanRNNCell_0']['Dense_Out']['kernel'].shape == (256, 10)
# assert params['params']['RNNLayer_0']['ScanRNNCell_0']['Dense_Out']['bias'].shape == (10,)
# assert params['params']['RNNLayer_0']['ScanRNNCell_0']['Dense_RNN']['kernel'].shape == (257, 256)
# assert params['params']['RNNLayer_0']['ScanRNNCell_0']['Dense_RNN']['bias'].shape == (256,)


params: 
 {'rnn_cells_0': {'SimpleScan': {'ScanIRRNCell_0': {'Whx': {'bias': (512,), 'kernel': (1, 512)}, 'Woh': {'bias': (10,), 'kernel': (512, 10)}, 'wh': (2, 512)}}}}


In [9]:
plot_dynamics(srn, params, batch_x, batch_y, dataset_version=DATASET_VERSION, id_sample=2)

In [10]:
key = jax.random.PRNGKey(0)
key, subkey = jax.random.split(key)
lr = 1e-3

In [11]:
@jax.jit
def update_model(state, grads):
    return state.apply_gradients(grads=grads)

In [12]:
def apply_model(state, images, labels):
    """Computes gradients, loss and accuracy for a single batch."""

    def loss_fn(params):
        _, out_hist = state.apply_fn({'params': params}, images)
        logits = out_hist.mean(axis=1)
        one_hot = jax.nn.one_hot(labels, 10)
        loss = jnp.mean(optax.softmax_cross_entropy(logits=logits, labels=one_hot))
        return loss, logits

    grad_fn = jax.value_and_grad(loss_fn, has_aux=True)
    (loss, logits), grads = grad_fn(state.params)
    accuracy = jnp.mean(jnp.argmax(logits, -1) == labels)
    return grads, loss, accuracy
    

In [13]:
def run_epoch(state, train_dl, rng):
    """Train for a single epoch."""

    epoch_loss = []
    epoch_accuracy = []

    progress_bar = tqdm(train_dl, desc="Training", leave=True)
    batch_id = 0
    for batch_images, batch_labels in progress_bar:
        grads, loss, accuracy = apply_model(state, batch_images, batch_labels)
        # print(jnp.max(grads['dense_0']['kernel']), jnp.min(grads['dense_0']['kernel']))
        # print(loss)
        state = update_model(state, grads)
        epoch_loss.append(loss)
        epoch_accuracy.append(accuracy)
        batch_id += 1
        if batch_id % 3 == 0:
            progress_bar.set_postfix(loss=loss.item(), accuracy=accuracy.item())
        
    train_loss = np.mean(epoch_loss)
    train_accuracy = np.mean(epoch_accuracy)
    return state, train_loss, train_accuracy


In [14]:
def create_train_state(key, model_cls, lr, dataset_version):
    
    init_x = jnp.ones((128, 784, 1)) if dataset_version == "sequential" else jnp.ones((128, 28, 28))

    model = model_cls(hidden_size=512, output_size=10, n_layers=1)
    params = model.init(key, init_x)['params']
    
    # Debugging: Print parameter structure
    print("Initialized parameter structure:", jax.tree_util.tree_map(jnp.shape, params))

    optimizer = optax.adam(lr)
    return train_state.TrainState.create(
        apply_fn=model.apply,
        params=params,
        tx=optimizer,
    )


In [15]:
state = create_train_state(key, BatchRNN, lr, DATASET_VERSION)

Initialized parameter structure: {'rnn_cells_0': {'SimpleScan': {'ScanIRRNCell_0': {'Whx': {'bias': (512,), 'kernel': (1, 512)}, 'Woh': {'bias': (10,), 'kernel': (512, 10)}, 'wh': (2, 512)}}}}


In [16]:
jax.devices()

[cuda(id=0)]

In [ ]:
for epoch in range(3):
    state, train_loss, train_accuracy = run_epoch(state, trainloader, key)
    print(f"Epoch {epoch} | Loss: {train_loss} | Accuracy: {train_accuracy}")

Training: 100%|██████████| 468/468 [02:23<00:00,  3.26it/s, accuracy=0.281, loss=1.97] 


Epoch 0 | Loss: 2.28014874458313 | Accuracy: 0.15306156873703003


Training: 100%|██████████| 468/468 [02:20<00:00,  3.34it/s, accuracy=0.258, loss=1.99]


Epoch 1 | Loss: 2.032831907272339 | Accuracy: 0.23753005266189575


Training: 100%|██████████| 468/468 [02:20<00:00,  3.33it/s, accuracy=0.312, loss=1.87]

Epoch 2 | Loss: 1.8504436016082764 | Accuracy: 0.3020332455635071


In [58]:
print(jax.tree_util.tree_map(jnp.shape, params['params']))
print(jax.tree_util.tree_map(jnp.shape, state.params))

{'rnn_cells_0': {'SimpleScan': {'ScanIRRNCell_0': {'Whx': {'bias': (512,), 'kernel': (28, 512)}, 'Woh': {'bias': (10,), 'kernel': (512, 10)}, 'wh': (2, 512)}}}}
{'rnn_cells_0': {'SimpleScan': {'ScanIRRNCell_0': {'Whx': {'bias': (512,), 'kernel': (28, 512)}, 'Woh': {'bias': (10,), 'kernel': (512, 10)}, 'wh': (2, 512)}}}}


In [59]:
fig, axs = plt.subplots(len(all_state_hist)+1, 3, figsize=(1200*px, (300+100*len(all_state_hist))*px))
axs[0,0].set_title('Whx Kernel')
axs[0,1].set_title('wh')
axs[0,2].set_title('Woh kernel')
for l in range(len(all_state_hist)):
    min_whx_init = params['params'][f'rnn_cells_{l}']['SimpleScan']['ScanIRRNCell_0']['Whx']['kernel'].min()
    min_whx = state.params[f'rnn_cells_{l}']['SimpleScan']['ScanIRRNCell_0']['Whx']['kernel'].min()
    min_whx = min(min_whx_init, min_whx)

    min_wh_init = params['params'][f'rnn_cells_{l}']['SimpleScan']['ScanIRRNCell_0']['wh'].min()
    min_wh = state.params[f'rnn_cells_{l}']['SimpleScan']['ScanIRRNCell_0']['wh'].min()
    min_wh = min(min_wh_init, min_wh)

    min_woh_init = params['params'][f'rnn_cells_{l}']['SimpleScan']['ScanIRRNCell_0']['Woh']['kernel'].min()
    min_woh = state.params[f'rnn_cells_{l}']['SimpleScan']['ScanIRRNCell_0']['Woh']['kernel'].min()
    min_woh = min(min_woh_init, min_woh)

    max_whx_init = params['params'][f'rnn_cells_{l}']['SimpleScan']['ScanIRRNCell_0']['Whx']['kernel'].max()
    max_whx = state.params[f'rnn_cells_{l}']['SimpleScan']['ScanIRRNCell_0']['Whx']['kernel'].max()
    max_whx = max(max_whx_init, max_whx)

    max_wh_init = params['params'][f'rnn_cells_{l}']['SimpleScan']['ScanIRRNCell_0']['wh'].max()
    max_wh = state.params[f'rnn_cells_{l}']['SimpleScan']['ScanIRRNCell_0']['wh'].max()
    max_wh = max(max_wh_init, max_wh)

    max_woh_init = params['params'][f'rnn_cells_{l}']['SimpleScan']['ScanIRRNCell_0']['Woh']['kernel'].max()
    max_woh = state.params[f'rnn_cells_{l}']['SimpleScan']['ScanIRRNCell_0']['Woh']['kernel'].max()
    max_woh = max(max_woh_init, max_woh)



    axs[l,0].hist(params['params'][f'rnn_cells_{l}']['SimpleScan']['ScanIRRNCell_0']['Whx']['kernel'].flatten(), bins=100, alpha=0.5, range=(min_whx, max_whx))
    axs[l,1].hist(params['params'][f'rnn_cells_{l}']['SimpleScan']['ScanIRRNCell_0']['wh'].flatten(), bins=100, alpha=0.5, range=(min_wh, max_wh))
    axs[l,2].hist(params['params'][f'rnn_cells_{l}']['SimpleScan']['ScanIRRNCell_0']['Woh']['kernel'].flatten(), bins=100, alpha=0.5, range=(min_woh, max_woh))
    axs[l,0].hist(state.params[f'rnn_cells_{l}']['SimpleScan']['ScanIRRNCell_0']['Whx']['kernel'].flatten(), bins=100, alpha=0.5, range=(min_whx, max_whx))
    axs[l,1].hist(state.params[f'rnn_cells_{l}']['SimpleScan']['ScanIRRNCell_0']['wh'].flatten(), bins=100, alpha=0.5, range=(min_wh, max_wh))
    axs[l,2].hist(state.params[f'rnn_cells_{l}']['SimpleScan']['ScanIRRNCell_0']['Woh']['kernel'].flatten(), bins=100, alpha=0.5, range=(min_woh, max_woh))

    # y axis title
    axs[l,0].set_ylabel(f'L{l+1}: [{min_whx:.2f}, {max_whx:.2f}]')
    axs[l,1].set_ylabel(f'L{l+1}: [{min_wh:.2f}, {max_wh:.2f}]')
    axs[l,2].set_ylabel(f'L{l+1}: [{min_woh:.2f}, {max_woh:.2f}]')
    # set y range
plt.tight_layout()
plt.show()

In [60]:
jnp.min(params['params'][f'rnn_cells_{l}']['SimpleScan']['ScanIRRNCell_0']['wh'])

Array(-0.999, dtype=float32)

In [66]:
plot_dynamics(srn, state.params, batch_x[:5], batch_y[:5], DATASET_VERSION,
              id_sample=1, nb_inputs_to_plot=5, nb_components_to_plot=5)

In [62]:
all_state_hist, out_hist = srn.apply({'params':state.params}, batch_x[:5])

In [63]:
y = out_hist.mean(axis=1)
print(y.shape)
assert y.shape == (5, 10)
print(y)

(5, 10)
[[ -6.216  -1.211  -1.125  -0.8     0.034  -1.266  -2.528   3.766  -3.095
    0.643]
 [ -4.089  -6.315   2.113  -1.173  -1.603   0.672   1.6    -5.205   0.962
   -2.039]
 [ -7.203   4.768  -2.155  -3.021   1.732  -3.139  -1.211   2.077  -4.429
    0.252]
 [  1.786 -13.406  -1.735  -1.999  -2.453  -1.324   1.953  -6.62    1.501
   -1.247]
 [ -1.437  -4.734  -2.553  -6.313   3.797  -1.555  -2.618   0.031  -2.451
    1.347]]


In [64]:
one_hot = jax.nn.one_hot(batch_y[:5], 10)
loss = optax.softmax_cross_entropy(logits=y, labels=one_hot)
print(loss.shape)
print(loss)
print(loss.mean())

(5,)
[0.097 0.803 0.123 1.13  0.118]
0.45413947
